In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [2]:
dataset=pd.read_csv("Social_Network_Ads.csv")

In [3]:
dataset

,User ID,Gender,Age,EstimatedSalary,Purchased
0,15624510,Male,19,19000,0
1,15810944,Male,35,20000,0
2,15668575,Female,26,43000,0
3,15603246,Female,27,57000,0
4,15804002,Male,19,76000,0
...,...,...,...,...,...
395,15691863,Female,46,41000,1
396,15706071,Male,51,23000,1
397,15654296,Female,50,20000,1
398,15755018,Male,36,33000,0


In [4]:
dataset=pd.get_dummies(dataset,dtype=int,drop_first=True)

In [5]:
dataset

,User ID,Age,EstimatedSalary,Purchased,Gender_Male
0,15624510,19,19000,0,1
1,15810944,35,20000,0,1
2,15668575,26,43000,0,0
3,15603246,27,57000,0,0
4,15804002,19,76000,0,1
...,...,...,...,...,...
395,15691863,46,41000,1,0
396,15706071,51,23000,1,1
397,15654296,50,20000,1,0
398,15755018,36,33000,0,1


In [6]:
dataset.columns

Index(['User ID', 'Age', 'EstimatedSalary', 'Purchased', 'Gender_Male'], dtype='object')

In [7]:
independant=dataset[['User ID','Age', 'EstimatedSalary',  'Gender_Male']]

In [8]:
independant

,User ID,Age,EstimatedSalary,Gender_Male
0,15624510,19,19000,1
1,15810944,35,20000,1
2,15668575,26,43000,0
3,15603246,27,57000,0
4,15804002,19,76000,1
...,...,...,...,...
395,15691863,46,41000,0
396,15706071,51,23000,1
397,15654296,50,20000,0
398,15755018,36,33000,1


In [9]:
independant.shape

(400, 4)

In [10]:
dependant= dataset[["Purchased"]]

In [11]:
dependant

,Purchased
0,0
1,0
2,0
3,0
4,0
...,...
395,1
396,1
397,1
398,0


In [12]:
dataset[["Purchased"]].value_counts()

Purchased
0            257
1            143
Name: count, dtype: int64

In [13]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(independant, dependant, test_size = 1/3, random_state = 0)


In [14]:
X_train

,User ID,Age,EstimatedSalary,Gender_Male
218,15666675,46,96000,0
101,15593014,28,59000,1
311,15622585,39,96000,1
194,15689751,28,89000,1
326,15713463,41,72000,1
...,...,...,...,...
323,15619465,48,30000,0
192,15779581,29,43000,1
117,15591433,36,52000,1
47,15776348,27,54000,0


In [15]:
from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
X_train = sc.fit_transform(X_train)
X_test = sc.transform(X_test)

In [16]:
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import GridSearchCV

param_grid = {'solver':['newton-cg', 'lbfgs', 'liblinear', 'saga'],
             'penalty':['l2']}
grid = GridSearchCV(LogisticRegression(), param_grid, refit = True, verbose = 3,n_jobs=-1,scoring='f1_weighted')

In [17]:
#Fitting the Model of Grid Search
grid.fit(X_train, y_train)

Fitting 5 folds for each of 4 candidates, totalling 20 fits


C:\Anaconda3\Lib\site-packages\sklearn\utils\validation.py:1300: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


GridSearchCV(estimator=LogisticRegression(), n_jobs=-1,
             param_grid={'penalty': ['l2'],
                         'solver': ['newton-cg', 'lbfgs', 'liblinear', 'saga']},
             scoring='f1_weighted', verbose=3)

In [18]:
# print best parameter after tuning 
#print(grid.best_params_) 
re=grid.cv_results_
#print(re)

grid_predictions = grid.predict(X_test)

from sklearn.metrics import confusion_matrix
cm = confusion_matrix(y_test,grid_predictions)



# print classification report 
from sklearn.metrics import classification_report
clf_report = classification_report(y_test, grid_predictions )



In [19]:
print("The confussion Matrix:\n",cm)

The confussion Matrix:
 [[80  5]
 [13 36]]


In [24]:
print("The report: \n",clf_report)

The report: 
               precision    recall  f1-score   support

           0       0.86      0.94      0.90        85
           1       0.88      0.73      0.80        49

    accuracy                           0.87       134
   macro avg       0.87      0.84      0.85       134
weighted avg       0.87      0.87      0.86       134



In [25]:
table=pd.DataFrame.from_dict(re)
table

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_penalty,param_solver,params,split0_test_score,split1_test_score,split2_test_score,split3_test_score,split4_test_score,mean_test_score,std_test_score,rank_test_score
0,0.027983,0.004048,0.021482,0.001785,l2,newton-cg,"{'penalty': 'l2', 'solver': 'newton-cg'}",0.822092,0.790949,0.698235,0.92351,0.862442,0.819446,0.075085,2
1,0.021187,0.005452,0.027182,0.005667,l2,lbfgs,"{'penalty': 'l2', 'solver': 'lbfgs'}",0.822092,0.790949,0.698235,0.92351,0.862442,0.819446,0.075085,2
2,0.028382,0.030717,0.022386,0.003199,l2,liblinear,"{'penalty': 'l2', 'solver': 'liblinear'}",0.822092,0.790949,0.698235,0.92351,0.883278,0.823613,0.077882,1
3,0.014079,0.001961,0.024586,0.003379,l2,saga,"{'penalty': 'l2', 'solver': 'saga'}",0.822092,0.790949,0.698235,0.92351,0.862442,0.819446,0.075085,2


In [21]:
from sklearn.metrics import f1_score
f1_macro=f1_score(y_test,grid_predictions,average='weighted')
print("The f1_macro value for best parameter {}:".format(grid.best_params_),f1_macro)

The f1_macro value for best parameter {'penalty': 'l2', 'solver': 'liblinear'}: 0.8627201073285259


In [22]:
from sklearn.metrics import roc_auc_score

roc_auc_score(y_test,grid.predict_proba(X_test)[:,1])


0.9457382953181273

In [ ]:
User_ID_input=float(input("User ID:"))
Age_input=float(input("Age:"))
Estimatedsalary_input=float(input("Estimatedsalary:"))
Gender_Male_input=int(input("Gender_Male 0 or 1:"))

In [ ]:
Future_Prediction=grid.predict([[User_ID_input,Age_input,Estimatedsalary_input,Gender_Male_input]])# change the paramter,play with it.
print("Future_Prediction={}".format(Future_Prediction))